**Домашнее задание "Введение в Feature Selection"**

In [1]:
# Common imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import random
from sklearn.linear_model import LinearRegression
import math

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize'] = (10, 5)

In [2]:
import os
import urllib
import shutil

def download_file(url, dir_path="data"):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
    file_name = os.path.split(url)[-1]
    file_path = os.path.join(dir_path, file_name)
    
    with urllib.request.urlopen(url) as response, open(file_path, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        
    return file_path

In [3]:
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler

In [4]:
download_file("http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv")
adv_df = pd.read_csv('data/Advertising.csv', usecols=[1,2,3,4])
adv_df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [5]:
adv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
TV           200 non-null float64
radio        200 non-null float64
newspaper    200 non-null float64
sales        200 non-null float64
dtypes: float64(4)
memory usage: 6.3 KB


**1.Разделить дата сет на трейн и тест в отношение 50:50 70:30 80:20 (с перемешиванием)**

In [59]:
# преобразуем данные и записываем результат в новую колонку 'log_tv'
adv_df['log_tv'] = adv_df.TV.apply(lambda x: math.pow(x, 0.4))
adv_df.head()

,TV,radio,newspaper,sales,pow_tv,log_tv
0,230.1,37.8,69.2,22.1,8.805756,8.805756
1,44.5,39.3,45.1,10.4,4.563983,4.563983
2,17.2,45.9,69.3,9.3,3.120408,3.120408
3,151.5,41.3,58.5,18.5,7.450151,7.450151
4,180.8,10.8,58.4,12.9,7.996121,7.996121


In [29]:
from sklearn.model_selection import train_test_split
import sklearn

In [30]:
# Разделим датасет на трейн и тест 50:50
adv_train_50x50, adv_test_50x50 = train_test_split(adv_df, test_size=0.5, random_state=42, shuffle=True)

print("Total transactions in train dataset: ", len(adv_train_50x50))
print("Total transactions in test dataset: ", len(adv_test_50x50))


Total transactions in train dataset:  100
Total transactions in test dataset:  100


In [31]:
# Разделим датасет на трейн и тест 70:30
adv_train_70x30, adv_test_70x30 = train_test_split(adv_df, test_size=0.3, random_state=42, shuffle=True)

print("Total transactions in train dataset: ", len(adv_train_70x30))
print("Total transactions in test dataset: ", len(adv_test_70x30))

Total transactions in train dataset:  140
Total transactions in test dataset:  60


In [32]:
# Разделим датасет на трейн и тест 80:20
adv_train_80x20, adv_test_80x20 = train_test_split(adv_df, test_size=0.2, random_state=42, shuffle=True)

print("Total transactions in train dataset: ", len(adv_train_80x20))
print("Total transactions in test dataset: ", len(adv_test_80x20))

Total transactions in train dataset:  160
Total transactions in test dataset:  40


**2.Обучать наши модели на трейне. Предсказывать и замерять метрику R^2 и на трейне и на тесте. 3.Проверить следующие модели, для каждого разделения: а) sales ~ log_tv + radio б) sales ~ TV + radio в) sales ~ TV + radio + newspaper**

**1) Модель 50х50**

In [40]:
#модель 50:50 для sales ~ log_tv + radio
two_x_lm_50x50_log_tv = smf.ols('sales ~ log_tv + radio', adv_train_50x50).fit()

#расчет метрик
print("RSS:", np.sum(two_x_lm_50x50_log_tv.resid ** 2))
print("RSE:", np.sqrt(np.sum(two_x_lm_50x50_log_tv.resid ** 2)) / (adv_train_50x50.shape[0] - 2 - 1))
print("R^2:", two_x_lm_50x50_log_tv.rsquared)

RSS: 200.23499560383388
RSE: 0.14588084055549225
R^2: 0.9273049082016976


In [41]:
#получение предсказания на тесте 50:50
y_pred_test_50x50 = two_x_lm_50x50_log_tv.predict(adv_test_50x50[['log_tv', 'radio']])
y_test_50x50 = adv_test_50x50['sales']

#расчет метрик
TSS_test = np.sum((y_test_50x50 - y_test_50x50.mean())**2)
RSS_test = np.sum((y_test_50x50 - y_pred_test_50x50)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_50x50.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_50x50, y_pred_test_50x50)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 180.2527057870786
RSE_test: 1.3631857237341372
R^2_test: 0.9310741259261296


In [42]:
#модель 50:50 для sales ~ TV + radio
two_x_lm_50x50 = smf.ols('sales ~ TV + radio', adv_train_50x50).fit()

#расчет метрик
print("RSS:", np.sum(two_x_lm_50x50.resid ** 2))
print("RSE:", np.sqrt(np.sum(two_x_lm_50x50.resid ** 2)) / (adv_train_50x50.shape[0] - 2 - 1))
print("R^2:", two_x_lm_50x50.rsquared)

RSS: 269.79672077541716
RSE: 0.16933494815087197
R^2: 0.9020506014720118


In [43]:
#получение предсказания на тесте 50:50
y_pred_test_50x50 = two_x_lm_50x50.predict(adv_test_50x50[['TV', 'radio']])
y_test_50x50 = adv_test_50x50['sales']

#расчет метрик
TSS_test = np.sum((y_test_50x50 - y_test_50x50.mean())**2)
RSS_test = np.sum((y_test_50x50 - y_pred_test_50x50)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_50x50.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_50x50, y_pred_test_50x50)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 306.90663871598946
RSE_test: 1.7787596707099644
R^2_test: 0.8826436017134698


In [44]:
#модель 50:50 для sales ~ TV + radio + newspaper
three_x_lm_50x50 = smf.ols('sales ~ TV + radio + newspaper', adv_train_50x50).fit()

#расчет метрик
print("RSS:", np.sum(three_x_lm_50x50.resid ** 2))
print("RSE:", np.sqrt(np.sum(three_x_lm_50x50.resid ** 2)) / (adv_train_50x50.shape[0] - 3 - 1))
print("R^2:", three_x_lm_50x50.rsquared)

RSS: 263.7072834762901
RSE: 0.1691569435694473
R^2: 0.9042613648908893


In [45]:
#получение предсказания на тесте 50:50
y_pred_test_50x50 = three_x_lm_50x50.predict(adv_test_50x50[['TV', 'radio', 'newspaper']])
y_test_50x50 = adv_test_50x50['sales']

#расчет метрик
TSS_test = np.sum((y_test_50x50 - y_test_50x50.mean())**2)
RSS_test = np.sum((y_test_50x50 - y_pred_test_50x50)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_50x50.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_50x50, y_pred_test_50x50)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 334.4786637735287
RSE_test: 1.856942015020873
R^2_test: 0.8721004816045134


**2) Модель 70х30**

In [46]:
#модель 70:30 для sales ~ log_tv + radio
two_x_lm_70x30_log_tv = smf.ols('sales ~ log_tv + radio', adv_train_70x30).fit()

#расчет метрик
print("RSS:", np.sum(two_x_lm_70x30_log_tv.resid ** 2))
print("RSE:", np.sqrt(np.sum(two_x_lm_70x30_log_tv.resid ** 2)) / (adv_train_70x30.shape[0] - 2 - 1))
print("R^2:", two_x_lm_70x30_log_tv.rsquared)

RSS: 256.44355253955246
RSE: 0.11688945268659374
R^2: 0.9301954868019677


In [47]:
#получение предсказания на тесте 70:30
y_pred_test_70x30 = two_x_lm_70x30_log_tv.predict(adv_test_70x30[['log_tv', 'radio']])
y_test_70x30 = adv_test_70x30['sales']

#расчет метрик
TSS_test = np.sum((y_test_70x30 - y_test_70x30.mean())**2)
RSS_test = np.sum((y_test_70x30 - y_pred_test_70x30)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_70x30.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_70x30, y_pred_test_70x30)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 119.3572825998626
RSE_test: 1.4470616483179553
R^2_test: 0.9271446043449548


In [48]:
#модель 70:30 для sales ~ TV + radio
two_x_lm_70x30 = smf.ols('sales ~ TV + radio', adv_train_70x30).fit()

#расчет метрик
print("RSS:", np.sum(two_x_lm_70x30.resid ** 2))
print("RSE:", np.sqrt(np.sum(two_x_lm_70x30.resid ** 2)) / (adv_train_70x30.shape[0] - 2 - 1))
print("R^2:", two_x_lm_70x30.rsquared)

RSS: 349.6011203718844
RSE: 0.13647900283453715
R^2: 0.9048377867980043


In [49]:
#получение предсказания на тесте 70:30
y_pred_test_70x30 = two_x_lm_70x30.predict(adv_test_70x30[['TV', 'radio']])
y_test_70x30 = adv_test_70x30['sales']

#расчет метрик
TSS_test = np.sum((y_test_70x30 - y_test_70x30.mean())**2)
RSS_test = np.sum((y_test_70x30 - y_pred_test_70x30)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_70x30.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_70x30, y_pred_test_70x30)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 220.14282327184094
RSE_test: 1.9652365746247507
R^2_test: 0.8656253548947074


In [50]:
#модель 70:30 для sales ~ TV + radio + newspaper
three_x_lm_70x30 = smf.ols('sales ~ TV + radio + newspaper', adv_train_70x30).fit()

#расчет метрик
print("RSS:", np.sum(three_x_lm_70x30.resid ** 2))
print("RSE:", np.sqrt(np.sum(three_x_lm_70x30.resid ** 2)) / (adv_train_70x30.shape[0] - 3 - 1))
print("R^2:", three_x_lm_70x30.rsquared)

RSS: 347.1097250468102
RSE: 0.13699177170572718
R^2: 0.9055159502227753


In [51]:
#получение предсказания на тесте 70:30
y_pred_test_70x30 = three_x_lm_70x30.predict(adv_test_70x30[['TV', 'radio', 'newspaper']])
y_test_70x30 = adv_test_70x30['sales']

#расчет метрик
TSS_test = np.sum((y_test_70x30 - y_test_70x30.mean())**2)
RSS_test = np.sum((y_test_70x30 - y_pred_test_70x30)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_70x30.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_70x30, y_pred_test_70x30)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 227.80783420291309
RSE_test: 1.999156989890125
R^2_test: 0.8609466508230368


**3) Модель 80х20**

In [52]:
#модель 80:20 для sales ~ log_tv + radio
two_x_lm_80x20_log_tv = smf.ols('sales ~ log_tv + radio', adv_train_80x20).fit()

#расчет метрик
print("RSS:", np.sum(two_x_lm_80x20_log_tv.resid ** 2))
print("RSE:", np.sqrt(np.sum(two_x_lm_80x20_log_tv.resid ** 2)) / (adv_train_80x20.shape[0] - 2 - 1))
print("R^2:", two_x_lm_80x20_log_tv.rsquared)

RSS: 300.130315845859
RSE: 0.11034566599798143
R^2: 0.9276759564687794


In [53]:
#получение предсказания на тесте 80:20
y_pred_test_80x20 = two_x_lm_80x20_log_tv.predict(adv_test_80x20[['log_tv', 'radio']])
y_test_80x20 = adv_test_80x20['sales']

#расчет метрик
TSS_test = np.sum((y_test_80x20 - y_test_80x20.mean())**2)
RSS_test = np.sum((y_test_80x20 - y_pred_test_80x20)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_80x20.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_80x20, y_pred_test_80x20)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 72.51557727371339
RSE_test: 1.3999573089409256
R^2_test: 0.942563909350695


In [55]:
#модель 80:20 для sales ~ TV + radio
two_x_lm_80x20 = smf.ols('sales ~ TV + radio', adv_train_80x20).fit()

#расчет метрик
print("RSS:", np.sum(two_x_lm_80x20.resid ** 2))
print("RSE:", np.sqrt(np.sum(two_x_lm_80x20.resid ** 2)) / (adv_train_80x20.shape[0] - 2 - 1))
print("R^2:", two_x_lm_80x20.rsquared)

RSS: 433.2465274979225
RSE: 0.13257691007330222
R^2: 0.8955982149747163


In [56]:
#получение предсказания на тесте 80:20
y_pred_test_80x20 = two_x_lm_80x20.predict(adv_test_80x20[['TV', 'radio']])
y_test_80x20 = adv_test_80x20['sales']

#расчет метрик
TSS_test = np.sum((y_test_80x20 - y_test_80x20.mean())**2)
RSS_test = np.sum((y_test_80x20 - y_pred_test_80x20)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_80x20.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_80x20, y_pred_test_80x20)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 125.51792036273409
RSE_test: 1.841840445320886
R^2_test: 0.9005833101920356


In [57]:
#модель 80:20 для sales ~ TV + radio + newspaper
three_x_lm_80x20 = smf.ols('sales ~ TV + radio + newspaper', adv_train_80x20).fit()

#расчет метрик
print("RSS:", np.sum(three_x_lm_80x20.resid ** 2))
print("RSE:", np.sqrt(np.sum(three_x_lm_80x20.resid ** 2)) / (adv_train_80x20.shape[0] - 3 - 1))
print("R^2:", three_x_lm_80x20.rsquared)

RSS: 432.82070769302624
RSE: 0.13336117616296772
R^2: 0.8957008271017818


In [58]:
#получение предсказания на тесте 80:20
y_pred_test_80x20 = three_x_lm_80x20.predict(adv_test_80x20[['TV', 'radio', 'newspaper']])
y_test_80x20 = adv_test_80x20['sales']

#расчет метрик
TSS_test = np.sum((y_test_80x20 - y_test_80x20.mean())**2)
RSS_test = np.sum((y_test_80x20 - y_pred_test_80x20)**2)
RSE_test = np.sqrt(RSS_test / (adv_test_80x20.shape[0] - 2 - 1))
R_2_test = sklearn.metrics.r2_score(y_test_80x20, y_pred_test_80x20)

print("RSS_test:", RSS_test)
print("RSE_test:", RSE_test)
print("R^2_test:", R_2_test)

RSS_test: 126.96389415904423
RSE_test: 1.852419120742681
R^2_test: 0.8994380241009119
